# MySA - My Skincare Assistant #
### Final project | Skincare Ingredient Reader | Quyen


## 1 - Setting environment, libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pip
!pip install google-api-python-client
!pip install google-cloud-vision
!pip install google-cloud-language
!pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 370 kB 5.1 MB/s            
     |████████████████████████████████| 109 kB 55.6 MB/s            
     |████████████████████████████████| 45 kB 3.3 MB/s             
     |████████████████████████████████| 1.1 MB 52.9 MB/s            
     |████████████████████████████████| 4.1 MB 48.9 MB/s            
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.26.3
    Uninstalling google-api-core-1.26.3:
  

     |████████████████████████████████| 93 kB 1.4 MB/s             
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.3.2
    Uninstalling google-api-core-2.3.2:
      Successfully uninstalled google-api-core-2.3.2


     |████████████████████████████████| 50 kB 3.2 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149866 sha256=9fdbf23ff7a44f67d44e12c594f239e209d52e8386674a98e0fc02f238c860dd
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-levenshtein


In [ ]:
from google.cloud import vision
from google.cloud.vision_v1 import types
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from google.colab import files
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pathlib
import glob
import re
import os
import io

In [ ]:
uploaded = files.upload()
!export GOOGLE_APPLICATION_CREDENTIALS="/content/personal_service_account.json"
!gcloud auth activate-service-account --key-file /content/personal_service_account.json

Saving personal_service_account.json to personal_service_account.json
Activated service account credentials for: [personal-google-cloud@spartan-tesla-335705.iam.gserviceaccount.com]


To take a quick anonymous survey, run:
  $ gcloud survey



In [ ]:
# setup environment 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/personal_service_account.json"
print("Service Account Key: {}".format(os.environ["GOOGLE_APPLICATION_CREDENTIALS"]))

Service Account Key: /content/personal_service_account.json


## 2 - Dataset

In [ ]:
# skincare product dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Dataset/skincare_dataset_final.csv', sep = ';')

# extracting brands (drop duplicates)
brands = df['Brand'].drop_duplicates().reset_index(drop = True)
brands

0                      LA MER
1                       SK-II
2              DRUNK ELEPHANT
3                IT COSMETICS
4                      TATCHA
                ...          
137                  ST. IVES
138    LOVE BEAUTY AND PLANET
139                   THAYERS
140                    CERAVE
141                 ORGANIQUE
Name: Brand, Length: 142, dtype: object

In [ ]:
# ingredient dictionary (scraped from paula choice website)
dic = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Dataset/ingredient_dictionary_update.csv', sep = ';')
dic

,name,rating,categories,benefits,description
0,"Aqua, Eau, Water",Average,Solvent,Hydration,"Further, too much water can be a problem for s..."
1,"1, 2-Hexanediol",Good,"Preservative, Humectant",NaN,A synthetic preservative and moisture-binding ...
2,10-Hydroxydecanoic Acid,Good,"Emollient, Occlusive/Opacifying Agent",Hydration,A synthetic ingredient that functions as a ski...
3,4-T-Butylcyclohexanol,Good,Emollient,Hydration,A synthetic fatty alcohol that functions as an...
4,Acacia Farnesiana Extract,Worst,"Fragrance: Synthetic and Natural, Plant Extracts",NaN,A fragrant extract from a type of acacia tree....
...,...,...,...,...,...
1952,Bixa Orellana Seed Oil,Good,"Plant Extracts, Emollient, Humectant","Moisturizing, Hydration",NaN
1953,Isodecyl Isononanoate,Average,"Emollient, Texture Enhancer",NaN,Helps form a protective film on the surface of...
1954,Opuntia Ficus-Indica Fruit Extract,Good,Plant Extracts,"Soothing, Hydration","The extract of Prickly Pear, a cactus that’s n..."
1955,Oleth-3 Phosphate,Average,"Emulsifier, Surfactant, Irritant, Comedogenic ...",NaN,It works as an emulsifier helping water and oi...


## 3 - Main function

In [ ]:
# main function that takes image path and return product item match and its ingredient explanation
def detect_product(path):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()
    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    text_raw = texts[0].description
    text_lean = re.sub('\n', ' ', text_raw)
    brand_probas = []
    for i in brands:
      partial_ratio = fuzz.partial_ratio(i.lower(),text_raw.lower())
      brand_probas.append(partial_ratio)
    if max(brand_probas) > 70:
      des_probas = []
      max_index = brand_probas.index(max(brand_probas))
      brand = brands[max_index]
      df_filtered = df[df.Brand == brand]
      for des in df_filtered.Name:
        des_ratio = fuzz.token_set_ratio(text_lean, des)
        des_probas.append(des_ratio)
      if max(des_probas) > 50:
        des_index = des_probas.index(max(des_probas))
        result_line = df_filtered.iloc[des_index]
        print(f'Top match item found: {max(des_probas) * max(brand_probas)/100}%')
        print(f'Brand: {result_line.Brand}')
        print(f'Description: {result_line.Name}')
        print(f'Price: ${result_line.Price}')
        print(f'Rating: {result_line.Rank}/5')
        print('Skin Type:')
        skin = ['Sensitive', 'Dry', 'Normal', 'Combination', 'Body']
        skin_type = [i for i in skin if result_line[i]>0]
        print(f'Skin Type: {skin_type}')
        print(f'INGREDIENT DECODING:')
        decoding_dic = {}
        decoding_dic['Ingredient'] = []
        decoding_dic['% Match'] = []
        decoding_dic['Match Item'] = []
        decoding_dic['Index'] = []
        for i in map(lambda x: x.strip(), result_line.Ingredients.split(',')):
          item, proba, index = process.extractOne(i,dic.name)
          decoding_dic['Ingredient'].append(i)
          decoding_dic['% Match'].append(proba)
          decoding_dic['Match Item'].append(item)
          decoding_dic['Index'].append(index)
        decoding_df = pd.DataFrame.from_dict(decoding_dic)
        final_df = decoding_df.merge(dic, how = 'left', left_on = decoding_df['Index'], right_index = True)
        final_df.drop(columns = ['Index', 'name'], inplace = True)
        # for i in map(lambda x: x.strip(), result_line.Ingredients.split(',')):
        #   item, proba, indx = process.extractOne(i,dic.name)
        #   print(i, top_match)    
        
      else:
        print("Sorry, I couldn't find any match. Would you like to try again? Make sure brand name and product description are readable. Or you call fill such info in the below.")
    else:
      print("Sorry, I couldn't find any match. Would you like to try again? Make sure brand name and product description are readable. Or you call fill such info in the below.")


In [ ]:
# putting ingredient explanation into presentable dataframe
decoding_dic = {}
decoding_dic['Ingredient'] = []
decoding_dic['% Match'] = []
decoding_dic['Match Item'] = []
decoding_dic['Index'] = []
for i in map(lambda x: x.strip(), result_line.Ingredients.split(',')):
  item, proba, index = process.extractOne(i,dic.name)
  decoding_dic['Ingredient'].append(i)
  decoding_dic['% Match'].append(proba)
  decoding_dic['Match Item'].append(item)
  decoding_dic['Index'].append(index)
decoding_df = pd.DataFrame.from_dict(decoding_dic)
final_df = decoding_df.merge(dic, how = 'left', left_on = decoding_df['Index'], right_index = True)
final_df.drop(columns = ['Index', 'name'], inplace = True)
final_df

,Ingredient,% Match,Match Item,rating,categories,benefits,description
0,Paraffinum Liquidum,100,Paraffinum Liquidum,Good,Emollient,Hydration,The Latin name for emollient ingredient minera...
1,Triisononanoin,100,Triisononanoin,Average,Texture Enhancer,NaN,Víscosity contrilling. Help thicken up product...
2,Cetearyl Ethylhexanoate,100,Cetearyl Ethylhexanoate,Good,"Texture Enhancer, Emollient",Hydration,Oil-like liquid that functions as an emollient...
3,Isopropyl Myristate,100,Isopropyl Myristate,Good,"Emollient, Texture Enhancer",Hydration,Texture enhancer and emollient as used in cosm...
4,Retinyl Palmitate,100,Retinyl Palmitate,Best,Antioxidant,Anti-Aging,The safety of retinyl palmitate is supported b...
5,Helianthus Annuus Seed Oil,95,Helianthus Annuus (Sunflower) Seed Oil,Best,"Emollient, Plant Extracts, Antioxidant","Anti-Aging, Hydration",Helianthus annuus (sunflower) seed oil is a no...
6,Tocopheryl Acetate,100,Tocopheryl Acetate,Best,Antioxidant,"Anti-Aging, Soothing",NaN
7,Anthemis Nobilis Flower Oil,87,Anthemis Nobilis Flower Extract,Best,"Plant Extracts, Antioxidant","Soothing, Anti-Aging",NaN
8,Lavandula Angustifolia Oil,95,Lavandula Angustifolia,Worst,"Fragrance: Synthetic and Natural, Irritant, Pl...",NaN,NaN
9,Rosmarinus Officinalis Leaf Oil,100,Rosmarinus Officinalis Leaf Oil,Bad,"Plant Extracts, Irritant, Antioxidant",NaN,The essential oil coming from the leafs of the...


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/ bio_oil.jpeg'
result_line = detect_product(path)

Top match item found: 64.0%
Brand: BIO-OIL
Description: Skincare oil 60ml
Price: $8
Rating: 4.4/5
Skin Type:
Skin Type: ['Body']
Ingredients: Paraffinum Liquidum, Triisononanoin, Cetearyl Ethylhexanoate, Isopropyl Myristate, Retinyl Palmitate, Helianthus Annuus Seed Oil, Tocopheryl Acetate, Anthemis Nobilis Flower Oil, Lavandula Angustifolia Oil, Rosmarinus Officinalis Leaf Oil, Calendula Officinalis Extract, Glycine Soja Oil, Bisabolol, Tocopherol, Parfum, Alpha-Isomethyl Ionone, Amyl Cinnamal, Benzyl Salicylate, Citronellol, Coumarin, Eugenol, Farnesol, Geraniol, Hydroxycitronellal, Limonene, Linalool, Ci 26100


In [ ]:
for i in map(lambda x: x.strip(), result_line.Ingredients.split(',')):
  item, proba, indx = process.extractOne(i,dic.name)
  print(i, top_match)  


Label                                                Moisturizer
Brand                                                    BIO-OIL
Name                                           Skincare oil 60ml
Price                                                          8
Rank                                                         4.4
Sensitive                                                      0
Dry                                                            0
Normal                                                         0
Combination                                                    0
Oily                                                           0
Body                                                           1
Ingredients    Paraffinum Liquidum, Triisononanoin, Cetearyl ...
Name: 1480, dtype: object

In [ ]:
# match items from skincare product dataset with all testing images. Print % match, series (matched line from the dataframe)

path = pathlib.Path('/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img')
files = path.rglob('*.*')
for f in files:
  detect_product(f)

96 ANESSA Ultimate beauty sunscreen Outstanding UVprotection Radiant skin begins today perfect UV sunscreen skincare milk SPF50* +++++  
 Label                                                Sun protect
Brand                                                     ANESSA
Name             Perfect UV sunscreen skincare milk SPF50+ PA+++
Price                                                         18
Rank                                                         4.8
Sensitive                                                      1
Dry                                                            0
Normal                                                         1
Combination                                                    1
Oily                                                           1
Body                                                           1
Ingredients    Cyclopentasiloxane, Zinc Oxide, Diisopropyl Se...
Name: 1481, dtype: object
82 LA ROCHE-POSAY LABORATOIRE DERMATOLOGIQUE spf Very hi

In [ ]:
# match 
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/exfoliate_leclub.jpeg'
result, match_proba, text_extract = detect_product(path)
for i in map(lambda x: x.strip(), result.Ingredients.split(',')):
  top_match = process.extractOne(i,dic.name)
  print(top_match)

('Kaolin', 100, 915)
('Aqua, Water, Eau', 90, 1927)
('Maltodextrin', 100, 1041)
('Bentonite', 100, 197)
('Glycerin', 100, 725)
('Phenoxyethanol', 100, 1301)
('Xanthan Gum', 100, 1891)
('PPG-26-Buteth-26', 94, 1406)
('Ananas Sativus Fruit Extract', 95, 109)
('Caprylyl Glycol', 100, 329)
('Agastache Mexicana (Hyssop) Flower/Leaf/Stem Extract', 86, 44)
('PEG-40 Hydrogenated Castor Oil', 100, 1269)
('Chlorphenesin', 100, 404)
('Iron Oxides', 90, 879)
('Cyamopsis tetragonoloba (Guar) Gum', 100, 502)
('Subtilisin', 100, 1928)
('Lipase', 100, 1929)


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/ bio_oil.jpeg'
result, match_proba, text_extract = detect_product(path)
for i in map(lambda x: x.strip(), result.Ingredients.split(',')):
  top_match = process.extractOne(i,dic.name)
  print(i, top_match)

Paraffinum Liquidum ('Paraffinum Liquidum', 100, 1238)
Triisononanoin ('Triisononanoin', 100, 1930)
Cetearyl Ethylhexanoate ('Cetearyl Ethylhexanoate', 100, 380)
Isopropyl Myristate ('Isopropyl Myristate', 100, 895)
Retinyl Palmitate ('Retinyl Palmitate', 100, 1478)
Helianthus Annuus Seed Oil ('Helianthus Annuus (Sunflower) Seed Oil', 95, 790)
Tocopheryl Acetate ('Tocopheryl Acetate', 100, 1783)
Anthemis Nobilis Flower Oil ('Anthemis Nobilis Flower Extract', 87, 119)
Lavandula Angustifolia Oil ('Lavandula Angustifolia', 95, 966)
Rosmarinus Officinalis Leaf Oil ('Rosmarinus Officinalis Leaf Oil', 100, 1934)
Calendula Officinalis Extract ('Calendula Officinalis Flower Extract', 95, 305)
Glycine Soja Oil ('Glycine', 90, 750)
Bisabolol ('Bisabolol', 100, 228)
Tocopherol ('Tocopherol', 100, 1781)
Parfum ('Perfume, Fragrance, Parfum', 90, 1933)
Alpha-Isomethyl Ionone ('Alpha Isomethyl Ionone', 100, 77)
Amyl Cinnamal ('Amyl Cinnamate, Amyl Cinnamal', 90, 104)
Benzyl Salicylate ('Benzyl Salicy

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/ body_jergen.jpeg'
result, match_proba, text_extract = detect_product(path)
for i in map(lambda x: x.strip(), result.Ingredients.split(',')):
  top_match = process.extractOne(i,dic.name)
  print(i, top_match)

Water ('Water', 100, 1874)
glycerin ('Glycerin', 100, 725)
zea mays (corn) starch ('Zea mays starch, corn starch', 95, 484)
cetearyl alcohol ('Cetearyl Alcohol', 100, 379)
dihydroxyacetone ('Dihydroxyacetone', 100, 543)
mineral oil ('Mineral Oil', 100, 1106)
ethylhexyl isononanoate ('ethylhexyl isononanoate', 100, 1935)
propanediol ('Propanediol', 100, 1413)
cetearth-20 ('Ceteareth-20', 96, 376)
dimethicone ('Dimethicone', 100, 552)
steareth-2 ('Steareth-20', 95, 1710)
fragrance ('Fragrance', 100, 681)
octyldodecyl myristate ('Octyldodecyl Myristate', 100, 1168)
stearic acid ('Stearic Acid', 100, 1711)
petrolatum ('Petrolatum', 100, 1298)
hydroxyethyl acrylate/sodium acryloyldimethyl taurate copolymer ('Hydroxyethyl Acrylate/Sodium Acryloyl Dimethyl Taurate Copolymer', 99, 851)
isohexadecane ('Isohexadecane', 100, 888)
caprylyl glycol ('Caprylyl Glycol', 100, 329)
erythrulose ('Erythrulose', 100, 619)
bht ('BHT', 100, 220)
polysorbate 60 ('Polysorbate 60', 100, 1380)
butylene glycol ('

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/ toner_Thayer.jpeg'
result, match_proba, text_extract = detect_product(path)
for i in map(lambda x: x.strip(), result.Ingredients.split(',')):
  top_match = process.extractOne(i,dic.name)
  print(i, top_match)

Aqua/Water/Eau ('Aqua, Water, Eau', 95, 1927)
Glycerin ('Glycerin', 100, 725)
Hamamelis Virginiana (Witch Hazel) Bark/Leaf/Twig Extract* ('Hamamelis Virginiana', 90, 782)
Aloe Barbadensis Leaf Extract* ('Aloe Barbadensis Leaf Juice Extract', 95, 69)
Rosa Centifolia (Rose) Flower Water ('Rosa Centifolia', 90, 1491)
Fragrance ('Fragrance', 100, 681)
Phenoxyethanol ('Phenoxyethanol', 100, 1301)
Caprylyl Glycol ('Caprylyl Glycol', 100, 329)
Ethylhexylglycerin ('Ethylhexylglycerin', 100, 628)
Citric Acid ('Citric Acid', 100, 422)
Potassium Hydroxide ('Potassium Hydroxide', 100, 1396)


In [ ]:
dic = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Dataset/ingredient_dictionary_update.csv', sep = ';')
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/cam_anessa.jpg'
result, match_proba, text_extract = detect_product(path)
for i in map(lambda x: x.strip(), result.Ingredients.split(',')):
  top_match = process.extractOne(i,dic.name)
  print(i, top_match)

Cyclopentasiloxane ('Cyclopentasiloxane', 100, 507)
Zinc Oxide ('Zinc Oxide', 100, 1918)
Diisopropyl Sebacate ('Diisopropyl Sebacate', 100, 1939)
Methyl Methacrylate Crosspolymer ('Methyl Methacrylate Crosspolymer ', 100, 1940)
Water ('Deionized/Demineralized Water', 90, 525)
Cetyl Ethylhexanoate ('Cetearyl Ethylhexanoate', 93, 380)
Pentaerythrityl Tetraethylhexanoate ('Pentaerythrityl Tetraethylhexanoate', 100, 1941)
Triethylhexanoin ('Triethylhexanoin', 100, 1804)
Butylene Glycol ('Butylene Glycol', 100, 273)
Diethylhexyl Succinate ('Diethylhexyl Succinate', 100, 1942)
Titanium Dioxide ('Titanium Dioxide', 100, 1780)
Dimethicone ('Dimethicone', 100, 552)
Glycerin ('Glycerin', 100, 725)
Polymethylsilsesquioxane ('Polymethylsilsesquioxane', 100, 1373)
Dextrin Palmitate ('Dextrin Palmitate', 100, 1943)
PEG-9 Polydimethylsiloxyethyl Dimethicone ('Lauryl PEG-9 Polydimethylsiloxyethyl Dimethicone', 95, 964)
Ethylhexyl Triazone ('Ethylhexyl Triazone', 100, 1944)
Diethylamino Hydroxybenzoyl 

In [ ]:
dic = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Dataset/ingredient_dictionary_update.csv', sep = ';')
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/cam_hatomugi.jpg'
result, match_proba, text_extract = detect_product(path)
print(result.Brand, result.Name)
for i in map(lambda x: x.strip(), result.Ingredients.split(',')):
  top_match = process.extractOne(i,dic.name)
  print(i, top_match)

HATOMUGI UV Milky Gel SPF 31 PA+++ 250ml
Water ('Deionized/Demineralized Water', 90, 525)
Ethanol ('Ethanol', 100, 624)
Ethylhexyl Methoxycinnamate ('Ethylhexyl Methoxycinnamate', 100, 629)
BG ('Agar', 45, 43)
DPG ('PEG', 67, 1250)
Dimethicone ('Dimethicone', 100, 552)
Steareth-6 ('TEA', 90, 1745)
Octocrylene ('Octocrylene', 100, 1166)
Hexyl Diethylaminohydroxybenzoylbenzoate ('Hexyl Cinnamal', 86, 796)
Carbomer ('Carbomer', 100, 335)
Acrylates /​ Alkyl Acrylate (C10-30) C10- 30 Crosspolymer ('Acrylates/C10-30 Alkyl Acrylate Crosspolymer', 95, 30)
Mineral Oil ('Mineral Oil', 100, 1106)
Hematode Seed Extract ('Actinidia Chinensis (Kiwi Fruit) Extract', 86, 35)
Squalane ('Squalane', 100, 1702)
Sodium Hyaluronate ('Sodium Hyaluronate', 100, 1644)
Xanthan Gum ('Xanthan Gum', 100, 1891)
EDTA-4 Na ('EDTA', 90, 595)
Sodium Hydroxide ('Sodium Hydroxide', 100, 1646)
Tea ('TEA', 100, 1745)
BHT ('BHT', 100, 220)
Phenoxyethanol ('Phenoxyethanol', 100, 1301)
Methyl Paraben ('Methylparaben', 96, 109

In [ ]:
dic = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Dataset/ingredient_dictionary_update.csv', sep = ';')
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/cleansing_oil_huxley.jpeg'
result, match_proba, text_extract = detect_product(path)
print(result.Brand, result.Name)
for i in map(lambda x: x.strip(), result.Ingredients.split(',')):
  top_match = process.extractOne(i,dic.name)
  print(i, top_match)

HUXLEY Secret of Sahara cleansing oil: deep clean, deep moist
1 ('Blue 1', 90, 245)
2-Hexanediol ('1, 2-Hexanediol', 95, 0)
Bixa Orellana Seed Oil ('Almond Oil', 86, 67)
Calendula Officinalis Flower Extract ('Calendula Officinalis Flower Extract', 100, 305)
Calendula Officinalis Flower Oil ('Calendula Officinalis Flower Extract', 88, 305)
Caprylic/​Capric Triglyceride ('Caprylic/Capric Triglyceride', 98, 324)
Ethylhexylglycerin ('Ethylhexylglycerin', 100, 628)
Fragrance ('Fragrance', 100, 681)
Glycine Soja (Soybean) Oil ('Glycine Soja Oil (Soybean)', 95, 751)
Helianthus Annuus (Sunflower) Seed Oil ('Helianthus Annuus (Sunflower) Seed Oil', 100, 790)
Hippophae Rhamnoides Oil ('Hippophae Rhamnoides', 95, 804)
Hydrogenated Polyisobutene ('Hydrogenated Polyisobutene', 100, 832)
Isodecyl Isononanoate ('Isononyl Isononanoate', 86, 890)
Isopropyl Palmitate ('Isopropyl Palmitate', 100, 896)
Moringa Oleifera Seed Oil ('Moringa Oleifera Seed Oil', 100, 1112)
Opuntia Ficus-Indica Fruit Extract ('

In [ ]:
dic = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Dataset/ingredient_dictionary_update.csv', sep = ';')
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/serum_estee2.jpeg'
result, match_proba, text_extract = detect_product(path)
print(result.Brand, result.Name)
for i in map(lambda x: x.strip(), result.Ingredients.split(',')):
  top_match = process.extractOne(i,dic.name)
  print(i, top_match)

ESTÉE LAUDER Advanced Night Repair Synchronized Recovery Complex II
Water ('Aqua, Eau, Water', 90, 0)
Bifida Ferment Lysate ('Bifida Ferment Lysate', 100, 222)
Methyl Gluceth-20 ('Methyl Gluceth-20', 100, 1088)
Peg-75 ('PEG', 90, 1251)
Bis-Peg-18 Methyl Ether Dimethyl Silane ('Bis-PEG-18 Methyl Ether Dimethyl Silane', 100, 232)
Butylene Glycol ('Butylene Glycol', 100, 274)
Propanediol ('Propanediol', 100, 1414)
Cola Acuminata (Kola) Seed Extract ('Cola Acuminata Seed Extract', 95, 465)
Hydrolyzed Algin ('Algin', 90, 61)
Pantethine ('Pantethine', 100, 1231)
Caffeine ('Caffeine', 100, 295)
Lecithin ('Lecithin', 100, 971)
Tripeptide-32 ('Tripeptide-32', 100, 1821)
Ethylhexylglycerin ('Ethylhexylglycerin', 100, 629)
Sodium Rna ('Hydroxyethyl Acrylate/Sodium Acryloyl Dimethyl Taurate Copolymer', 86, 852)
Bisabolol ('Bisabolol', 100, 229)
Glycereth-26 ('Glycereth-26', 100, 723)
Squalane ('Squalane', 100, 1703)
Sodium Hyaluronate ('Sodium Hyaluronate', 100, 1645)
Oleth-3 Phosphate ('Oleth-3 P

In [ ]:
# testing accuracy in matching BRAND, put into csv file for accuracy evaluation and error analysis
path = pathlib.Path('/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img')
files = path.rglob('*.*')
test_list = []
test_product = {}
for f in files:
  test_product['img'] = str(f).split('/')[-1]
  test_product['text'], test_product['brand'], test_product['proba'] = detect_brand(f)
  test_list.append(test_product)
test_list.to_csv('test_list.csv')

In [ ]:
# testing accuracy in matching product description (brand must be matched previously)
for pack, brand in zip(test_list.text_lean, test_list.brand):
  des_probas = []
  df_filtered = df[df.Brand == brand.strip()]
  for des in df_filtered.Name:
    des_ratio = fuzz.token_set_ratio(pack, des)
    des_probas.append(des_ratio)
  if len(des_probas) < 1:
    print('sorry', pack)
  elif max(des_probas) > 50:
    des_index = des_probas.index(max(des_probas))
    print(max(des_probas),pack,'\n',df_filtered.iloc[des_index])

96 ANESSA Ultimate beauty sunscreen Outstanding UVprotection Radiant skin begins today perfect UV sunscreen skincare milk SPF50* +++++  
 Label                                                Sun protect
Brand                                                     ANESSA
Name             Perfect UV sunscreen skincare milk SPF50+ PA+++
Price                                                         18
Rank                                                         4.8
Sensitive                                                      1
Dry                                                            0
Normal                                                         1
Combination                                                    1
Oily                                                           1
Body                                                           1
Ingredients    Cyclopentasiloxane, Zinc Oxide, Diisopropyl Se...
Name: 1481, dtype: object
82 LA ROCHE-POSAY LABORATOIRE DERMATOLOGIQUE spf Very hi

In [ ]:
pack = 'Da telleis bLoqnncr FLO Som 6 2Eeare OAIO WICH NOUD J Bio-Oil® Skincarc Oil Dầu chăm sóc da 60ml'
des_probas = []
df_filtered = df[df.Brand == 'BIO-OIL']
for des in df_filtered.Name:
  des_ratio = fuzz.token_set_ratio(pack, des)
  des_probas.append(des_ratio)
if max(des_probas) > 60:
  des_index = des_probas.index(max(des_probas))
  print(max(des_probas),'\n',df_filtered.iloc[des_index])

64 
 Label                                                Moisturizer
Brand                                                    BIO-OIL
Name                                           Skincare oil 60ml
Price                                                          8
Rank                                                         4.4
Sensitive                                                      0
Dry                                                            0
Normal                                                         0
Combination                                                    0
Oily                                                           0
Body                                                           1
Ingredients    Paraffinum Liquidum, Triisononanoin, Cetearyl ...
Name: 1480, dtype: object


In [ ]:
# DR.JART+ inspect: lower proba 90 to 80
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/exfoliate_drjart.jpeg'
brand, _ = detect_brand(path)

('DR.JART+', 100)

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/remover_loreal.jpeg'
detect_brand(path)

('LOREAL DERMO-EXPERTISE GENTLE Lip and eye make-up remover For waterproof make-up Dermatologically and ophthalmologically tested ',
 'MAKE UP FOR EVER',
 75)

In [ ]:
proba_list = []
df_filtered = df[df.Brand == brand].Name
for des in df_filtered:
  token_set_ratio = fuzz.token_set_ratio(text_lean, des)
  proba_list.append(token_set_ratio)
if max(proba_list) > 80:
  max_index = proba_list.index(max(proba_list))
  product = df_filtered.iloc[max_index]
  print(brand, product, max(proba_list))
else:
  print("Sorry. I couldn't find anything that matches your image.")



DR.JART+ Dermaclear™ Micro Milk Peel 100


In [ ]:
# D PROGRAM, did not extract all text
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/toner_D.jpeg'
detect_text(path)

'I NOIO1 VITAL ACT program '

In [ ]:
# love planet and beauty, cant detect handwritten charaters well
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/wash_Unilever.jpeg'
detect_text(path)

"LOVE AND anet with COCONUT OIL & YLANG YLANG aroma tropical refresh BODY WASH IT'S VEGAN! 400ML "

In [ ]:
# Cleansing oil HUXLEY
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/gel_son&park.jpeg'
detect_text(path)

'BEAUTY GEL SON&PARK '

In [ ]:
# Cream CERAVE
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/cream_cerave.jpeg'
detect_brand(path)

CERAVE 100


In [ ]:
# Cream huxley
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/cream_huxley.jpeg'
detect_brand(path)

HUXLEY 100


In [ ]:
# function to read text from upload image
def detect_text(path):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()
    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    text_raw = texts[0].description
    return re.sub('\n', ' ', text_raw)

    # for text in texts:
    #     print('\n"{}"'.format(text.description))

    #     vertices = (['({},{})'.format(vertex.x, vertex.y)
    #                 for vertex in text.bounding_poly.vertices])

    #     print('bounds: {}'.format(','.join(vertices)))

    # if response.error.message:
    #     raise Exception(
    #         '{}\nFor more info on error messages, check: '
    #         'https://cloud.google.com/apis/design/errors'.format(
    #             response.error.message))
        

In [ ]:
def brand_matching(path):
  raw_text = detect_text(path)
  proba_list = []
  for i in brands:
    partial_ratio = fuzz.partial_ratio(i.lower(),text_raw.lower())
    proba_list.append(partial_ratio)
  if max(proba_list) > 85:
    max_index = proba_list.index(max(proba_list))
    print(brands[max_index], max(proba_list))
  else:
    print("Sorry. I couldn't find anything that matches your image.")

In [ ]:
brand_matching(path)

HATOMUGI 100


['Da telleis',
 'bLoqnncr',
 'FLO',
 'Som 6',
 '2Eeare OAIO WICH',
 'NOUD J',
 'Bio-Oil®',
 'Skincarc Oil',
 'Dầu chăm sóc da',
 '60ml',
 '']

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/sun_hatomugi.jpg'
text_raw = detect_text(path)
text_raw

'HATOM UGI\n*日\n1エSーキ1ミAN\nUV care & moisturizing\nSPF31\nPA+++\nThe UV milky gel\ncontaining coix seed extract\nfurnishes the skin with the\ndamage due to sunburn.\naとキ工キ7\n250ml 8.5 fl.oz.\n'

In [ ]:
# function to read text form online image:
def labelAnnotation(path):
  # Instantiates a client
  client = vision.ImageAnnotatorClient()
  with io.open(path, 'rb') as image_file:
      content = image_file.read()
  image = vision.Image(content = content)
  response = client.annotate_image({
     'image': image
  })
  # Detect and Print Labels:
  labels = response.label_annotations
  print("Labels: ")
  for label in labels:
    print("Description: ", label.description, "Confidence: ", label.score)

labelAnnotation('/content/drive/MyDrive/Colab Notebooks/AA - FINAL PROJECT/Photo raw/Front product img/cream_loreal.jpeg')

Labels: 
Description:  Fluid Confidence:  0.8588600754737854
Description:  Finger Confidence:  0.8410184979438782
Description:  Material property Confidence:  0.8013235330581665
Description:  Drink Confidence:  0.7873716950416565
Description:  Tints and shades Confidence:  0.7730051279067993
Description:  Nail Confidence:  0.7151855826377869
Description:  Wood Confidence:  0.6919101476669312
Description:  Thumb Confidence:  0.6813793182373047
Description:  Plastic Confidence:  0.655544638633728
Description:  Cylinder Confidence:  0.6490561962127686


In [ ]:
print(text_raw)

HATOM UGI
*日
1エSーキ1ミAN
UV care & moisturizing
SPF31
PA+++
The UV milky gel
containing coix seed extract
furnishes the skin with the
damage due to sunburn.
aとキ工キ7
250ml 8.5 fl.oz.

